In [0]:
!pip install tqdm
!pip install tqdm --upgrade

Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.45.0)


In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input,Embedding,Reshape, Concatenate
from tensorflow.keras.models import Sequential, Model
from keras.utils import plot_model
from IPython.display import Image
import tensorflow as tf
from tensorflow.keras.optimizers import Adam,SGD
from tqdm.keras import TqdmCallback

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cols = ['DURATION','COLOR','YEAR','DIRECTOR','RATING','COUNTRY_0','GENRES_0','GENRES_1','GENRES_2','LANGUAGE_0','KEYWORDS_0','KEYWORDS_1','KEYWORDS_2','WRITERS_0']
#data = pd.read_csv('/content/drive/My Drive/TFM/data/tabular_prueba_4.csv',header=0,decimal=',',usecols=cols)
data = pd.read_csv("/content/drive/My Drive/Máster KSchool/TFM/data/tabular_prueba_4.csv",header=0,decimal=',',usecols=cols)

data.head(5)

,DURATION,COLOR,YEAR,DIRECTOR,RATING,GENRES_0,GENRES_1,GENRES_2,LANGUAGE_0,KEYWORDS_0,KEYWORDS_1,KEYWORDS_2,COUNTRY_0,WRITERS_0
0,106,Color,2017,JohnDoe,5.3,Drama,Musical,Drama,English,poetry,college,rap,USA,JaneDoe
1,162,Color,2004,LawrenceRaghavendra,6.4,Action,Drama,Action,Telugu,mafia,friend,revenge,India,JaneDoe
2,85,Color,2009,MichaelHerbig,5.5,Action,Adventure,Comedy,German,hidinginabox,hidinginachest,vikingship,Germany,MichaelHerbig
3,83,Color,2017,IssaLÃƒÂ³pez,7.0,Fantasy,Horror,Mystery,Spanish,mexicanhorror,producedbydirector,titledirectedbyfemale,Mexico,IssaLÃƒÂ³pez
4,97,Color,2000,MikeFiggis,6.0,Drama,Romance,Drama,English,audition,jealousy,realtime,USA,MikeFiggis


In [0]:
data['RATING']  = data['RATING'].astype(float)
data['RATING_CAT'] = pd.cut(data['RATING'], bins = [0,5, 6.5,10], labels=['1','2','3'])
data.head(3)

,DURATION,COLOR,YEAR,DIRECTOR,RATING,GENRES_0,GENRES_1,GENRES_2,LANGUAGE_0,KEYWORDS_0,KEYWORDS_1,KEYWORDS_2,COUNTRY_0,WRITERS_0,RATING_CAT
0,106,Color,2017,JohnDoe,5.3,Drama,Musical,Drama,English,poetry,college,rap,USA,JaneDoe,2
1,162,Color,2004,LawrenceRaghavendra,6.4,Action,Drama,Action,Telugu,mafia,friend,revenge,India,JaneDoe,2
2,85,Color,2009,MichaelHerbig,5.5,Action,Adventure,Comedy,German,hidinginabox,hidinginachest,vikingship,Germany,MichaelHerbig,2


In [0]:
print(len(data[data['RATING_CAT']=='3']))
print(len(data[data['RATING_CAT']=='2']))
print(len(data[data['RATING_CAT']=='1']))

40270
47141
19770


In [0]:
data['KEYWORDS_1'] = data['KEYWORDS_1'].astype(str)
data['KEYWORDS_2'] = data['KEYWORDS_2'].astype(str)
data['WRITERS_0'] = data['WRITERS_0'].astype(str)

## Variables que se van a usar para el mlp

duracion--> directa

color --> directa

year --> int

director --> directa

generos --> solo el primero

language --> el primero

contry --> directo

keywords --> solo el primero

writers --> solo el primero


 ### funcion para coger el primer item





In [0]:
print(len(data.COLOR.unique()),
len(data.DIRECTOR.unique()),
len(data.GENRES_0.unique()),
len(data.GENRES_1.unique()),
len(data.GENRES_2.unique()),
len(data.LANGUAGE_0.unique()),
len(data.COUNTRY_0.unique()),
len(data.KEYWORDS_0.unique()),
len(data.KEYWORDS_1.unique()),
len(data.KEYWORDS_2.unique()))

2 6475 25 26 27 182 175 19511 20652 21665


In [0]:
data.COUNTRY_0.unique()

array(['USA', 'India', 'Germany', 'Mexico', 'Spain', 'Italy', 'France',
       'Yugoslavia', 'UK', 'Greece', 'Romania', 'Egypt', 'HongKong',
       'Brazil', 'China', 'Australia', 'Austria', 'Japan', 'Poland',
       'Russia', 'Venezuela', 'Czechoslovakia', 'Canada', 'SovietUnion',
       'Argentina', 'Turkey', 'WestGermany', 'Sweden', 'Finland',
       'Israel', 'Denmark', 'Colombia', 'CzechRepublic', 'ElSalvador',
       'SouthKorea', 'Hungary', 'Lithuania', 'Iran', 'Vietnam',
       'NewZealand', 'Philippines', "CÃƒÂ´ted'Ivoire", 'Portugal',
       'Belgium', 'Chile', 'SouthAfrica', 'Libya', 'Kazakhstan',
       'Switzerland', 'Netherlands', 'DominicanRepublic', 'Croatia',
       'Indonesia', 'Taiwan', 'Ireland', 'Norway', 'Bangladesh',
       'Zimbabwe', 'Peru', 'Jordan', 'Serbia', 'Kenya', 'Palestine',
       'Tunisia', 'EastGermany', 'Estonia', 'BurkinaFaso', 'Bulgaria',
       'Georgia', 'Slovakia', 'Iceland', 'Cuba', 'Panama', 'Slovenia',
       'Armenia', 'Thailand', 'UnitedAr

In [0]:
features = [0,1,2,3,5,6,7,8,9,10,11,12,13]
target = [14]
X = data.iloc[:,features]
y = data.iloc[:,target]

In [0]:
X

,DURATION,COLOR,YEAR,DIRECTOR,GENRES_0,GENRES_1,GENRES_2,LANGUAGE_0,KEYWORDS_0,KEYWORDS_1,KEYWORDS_2,COUNTRY_0,WRITERS_0
0,106,Color,2017,JohnDoe,Drama,Musical,Drama,English,poetry,college,rap,USA,JaneDoe
1,162,Color,2004,LawrenceRaghavendra,Action,Drama,Action,Telugu,mafia,friend,revenge,India,JaneDoe
2,85,Color,2009,MichaelHerbig,Action,Adventure,Comedy,German,hidinginabox,hidinginachest,vikingship,Germany,MichaelHerbig
3,83,Color,2017,IssaLÃƒÂ³pez,Fantasy,Horror,Mystery,Spanish,mexicanhorror,producedbydirector,titledirectedbyfemale,Mexico,IssaLÃƒÂ³pez
4,97,Color,2000,MikeFiggis,Drama,Romance,Drama,English,audition,jealousy,realtime,USA,MikeFiggis
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107176,114,Color,1991,JirÃƒÂ­Svoboda,Crime,Drama,Crime,Czech,basedonnovel,nan,basedonnovel,Czechoslovakia,JaneDoe
107177,83,Color,1959,JohnDoe,Adventure,Sci-Fi,Adventure,English,psychotronicfilm,notitleatbeginning,martian,USA,JaneDoe
107178,80,BlackandWhite,1961,PedroL.RamÃƒÂ­rez,Comedy,Comedy,Comedy,Spanish,basedonplay,nan,basedonplay,Spain,EnriqueJardielPoncela
107179,82,Color,2000,JohnDoe,Comedy,Comedy,Comedy,English,party,partliveaction,partanimation,USA,JaneDoe


In [0]:
y

,RATING_CAT
0,2
1,2
2,2
3,3
4,2
...,...
107176,3
107177,2
107178,2
107179,2


In [0]:
ency = OneHotEncoder(handle_unknown='ignore', sparse = False)
ency.fit(y.values)
y_one_hot = ency.transform(y.values)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2,stratify=y_one_hot)

In [0]:
categorical_vars = ['COLOR','DIRECTOR','GENRES_0','GENRES_1','GENRES_2','LANGUAGE_0','COUNTRY_0','KEYWORDS_0','KEYWORDS_1','KEYWORDS_2','WRITERS_0']
numeric_vars = ['DURATION','YEAR']

In [0]:
def preproc(X_train, X_test, categorical_vars=categorical_vars, numeric_vars=numeric_vars):
    
    """Asigna ids a la variables categoricas
    Return: Devuelve una lista por cada caloumna del dataframe con sus registros correspondientes"""
    input_list_train = []
    input_list_test = []

    for c in categorical_vars:
        print(c)

        raw_vals = np.unique(X_train[c])
        val_map = {}
        for i in range(len(raw_vals)):
            val_map[raw_vals[i]] = i       
        
        input_list_train.append(X_train[c].map(val_map).values)
    
        input_list_test.append(X_test[c].map(val_map).fillna(0).values)
        

    for n in numeric_vars:
    
      input_list_train.append(X_train[n].values)
      input_list_test.append(X_test[n].values)
    
    return input_list_train, input_list_test

In [0]:
input_list_train,input_list_test = preproc(X_train,X_test)

COLOR
DIRECTOR
GENRES_0
GENRES_1
GENRES_2
LANGUAGE_0
COUNTRY_0
KEYWORDS_0
KEYWORDS_1
KEYWORDS_2
WRITERS_0


In [0]:
input_list_train

[array([1, 1, 1, ..., 1, 1, 1]),
 array([2893, 4503, 2893, ...,  383, 3665, 3720]),
 array([ 3,  8, 13, ...,  6,  8, 13]),
 array([ 0, 19, 13, ...,  8,  8, 16]),
 array([20,  8, 13, ..., 23,  8, 23]),
 array([72, 42, 42, ..., 42, 42, 42]),
 array([ 76,  56, 158, ..., 158, 158, 158]),
 array([ 1286,  2029, 15328, ..., 10266, 15305,  7035]),
 array([ 3992, 11629, 10460, ...,  8090, 16192, 10714]),
 array([ 5921,  7003,   378, ..., 15538,  6348,   808]),
 array([2411, 2411, 2454, ..., 4050, 2411, 3355]),
 array([ 85, 128,  79, ..., 139,  79,  75]),
 array([2018, 2004, 2006, ..., 1999, 2008, 2017])]

In [0]:
y_train

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])

MODELO

https://datascience.stackexchange.com/questions/42730/confusion-about-entity-embeddings-of-categorical-variables-working-example


In [0]:
input_models=[]
output_embeddings=[]

for categorical_var in categorical_vars:
    
    cat_emb_name= categorical_var +'_Embedding'
    input_name= 'Input_' + categorical_var

    no_of_unique_cat  = X_train[categorical_var].nunique()
    
    embedding_size = int(min(np.ceil((no_of_unique_cat)/2), 100))  #Formula empírica

   
    input_model = Input(shape=(1,), name=input_name)
    
    output_model = Embedding(no_of_unique_cat, embedding_size, name=cat_emb_name)(input_model)
    output_model = Reshape(target_shape=(embedding_size,))(output_model)    

    input_models.append(input_model)

    output_embeddings.append(output_model)

In [0]:
for numeric_var in numeric_vars:
 
  input_numeric = Input(shape=(1,))
  dense_numeric = Dense(16)(input_numeric) 
  input_models.append(input_numeric)
  output_embeddings.append(dense_numeric)

In [0]:
output = Concatenate()(output_embeddings)
output = Dense(1024,activation='relu')(output)
output = Dense(512,activation='relu')(output)
output = Dense(256,activation='relu')(output)
output = Dense(3, activation='softmax')(output)

In [0]:
model = Model(inputs=input_models, outputs=output)

# Para probar con otro optimizador
sgd = SGD(lr=0.0000001)

model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_COLOR (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
Input_DIRECTOR (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
Input_GENRES_0 (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
Input_GENRES_1 (InputLayer)     [(None, 1)]          0                                            
______________________________________________________________________________________________

In [0]:
plot_model(model, to_file='model3.png',show_shapes=True,show_layer_names=True)
Image(retina=True, filename='model3.png')

TypeError: ignored

In [0]:
 hist = model.fit(input_list_train,y_train,validation_data=(input_list_test,y_test) , epochs = 200, batch_size = 128, verbose=2)

Epoch 1/200
670/670 - 7s - loss: 22.3226 - accuracy: 0.3381 - val_loss: 1.2324 - val_accuracy: 0.4078
Epoch 2/200
670/670 - 7s - loss: 1.1877 - accuracy: 0.3971 - val_loss: 1.1975 - val_accuracy: 0.4181
Epoch 3/200
670/670 - 7s - loss: 1.1617 - accuracy: 0.4007 - val_loss: 1.1819 - val_accuracy: 0.3697
Epoch 4/200
670/670 - 7s - loss: 1.1447 - accuracy: 0.4035 - val_loss: 1.1555 - val_accuracy: 0.4143
Epoch 5/200
670/670 - 7s - loss: 1.1231 - accuracy: 0.4122 - val_loss: 1.1447 - val_accuracy: 0.4289
Epoch 6/200
670/670 - 7s - loss: 1.1131 - accuracy: 0.4124 - val_loss: 1.1362 - val_accuracy: 0.4287
Epoch 7/200
670/670 - 7s - loss: 1.1066 - accuracy: 0.4131 - val_loss: 1.1294 - val_accuracy: 0.4161
Epoch 8/200
670/670 - 7s - loss: 1.1028 - accuracy: 0.4133 - val_loss: 1.1261 - val_accuracy: 0.4144
Epoch 9/200
670/670 - 7s - loss: 1.0994 - accuracy: 0.4121 - val_loss: 1.1228 - val_accuracy: 0.4157
Epoch 10/200
670/670 - 7s - loss: 1.0966 - accuracy: 0.4135 - val_loss: 1.1204 - val_accur

KeyboardInterrupt: ignored

loss suelo : 1.0219 val loss: 1.0211